<a href="https://colab.research.google.com/github/bobbyrayyy/mozillaTTS/blob/main/TTS_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hands-on example for TTS  [https://github.com/mozilla/TTS](https://github.com/mozilla/TTS)

This notebook trains Tacotron model on LJSpeech dataset.

In [1]:
# download LJSpeech dataset
!wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
# decompress
!tar -xjf LJSpeech-1.1.tar.bz2

--2021-01-08 07:46:25--  http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  48.8MB/s    in 54s     

2021-01-08 07:47:19 (48.2 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [2]:
# create train-val splits
!shuf LJSpeech-1.1/metadata.csv > LJSpeech-1.1/metadata_shuf.csv
!head -n 12000 LJSpeech-1.1/metadata_shuf.csv > LJSpeech-1.1/metadata_train.csv
!tail -n 1100 LJSpeech-1.1/metadata_shuf.csv > LJSpeech-1.1/metadata_val.csv

In [3]:
# get TTS to your local
!ls
!git clone https://github.com/mozilla/TTS

LJSpeech-1.1  LJSpeech-1.1.tar.bz2  sample_data
Cloning into 'TTS'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 10320 (delta 9), reused 12 (delta 4), pack-reused 10300
Receiving objects: 100% (10320/10320), 120.37 MiB | 31.54 MiB/s, done.
Resolving deltas: 100% (7185/7185), done.


In [4]:
# install espeak backend if you like to use phonemes instead of raw characters
!sudo apt-get install espeak
!pip install soundfile

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak amd64 1.48.04+dfsg-5 [61.6 kB]
Fetched 1,219 kB in 1s (901 kB/s)
deb

In [5]:
!cd
!ls


LJSpeech-1.1  LJSpeech-1.1.tar.bz2  sample_data  TTS


In [6]:
%cd TTS

/content/TTS


In [7]:
# install TTS requirements
!python setup.py install

Compiling ./TTS/tts/layers/glow_tts/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing ./TTS/tts/layers/glow_tts/monotonic_align/core.pyx
running install
running bdist_egg
running egg_info
creating TTS.egg-info
writing TTS.egg-info/PKG-INFO
writing dependency_links to TTS.egg-info/dependency_links.txt
writing entry points to TTS.egg-info/entry_points.txt
writing requirements to TTS.egg-info/requires.txt
writing top-level names to TTS.egg-info/top_level.txt
writing manifest file 'TTS.egg-info/SOURCES.txt'
writing manifest file 'TTS.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
-- Building version 0.0.6+9cf474a
creating temp_build
creating temp_build/TTS
copying TTS/__init__.py -> temp_build/TTS
creating temp_build/TTS/tts
copying TTS/tts/__init__.py -> temp_build/TTS/tts
creating temp_build/TTS/server
copying TTS/server/synthesizer.py -> temp_build/TTS/server
copying TTS/server/__init__.py -> temp_build/TTS

In [8]:

!ls


import json
from TTS.utils.io import load_config
CONFIG = load_config('/content/TTS/TTS/tts/configs/config.json')
CONFIG['datasets'][0]['path'] = '../LJSpeech-1.1/'
CONFIG['output_path'] = '../'
with open('/content/TTS/TTS/tts/configs/config.json', 'w') as fp:
    json.dump(CONFIG, fp)


build		    Dockerfile	 requirements_tests.txt  temp_build
CODE_OF_CONDUCT.md  images	 requirements.txt	 tests
CODE_OWNERS.rst     LICENSE.txt  run_tests.sh		 TTS
CONTRIBUTING.md     notebooks	 setup.cfg		 TTS.egg-info
dist		    README.md	 setup.py		 version.py


In [9]:

%cd
%cd /content/TTS/TTS/bin
!ls
!pip install -r /content/TTS/requirements.txt


/root
/content/TTS/TTS/bin
compute_embeddings.py		  distribute.py      train_vocoder_gan.py
compute_statistics.py		  distribute_tts.py  train_vocoder_wavegrad.py
convert_melgan_tflite.py	  synthesize.py      train_vocoder_wavernn.py
convert_melgan_torch_to_tf.py	  train_encoder.py   tune_wavegrad.py
convert_tacotron2_tflite.py	  train_glow_tts.py
convert_tacotron2_torch_to_tf.py  train_tacotron.py
     |████████████████████████████████| 320.4MB 50kB/s 
     |████████████████████████████████| 1.6MB 49.4MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 235kB 55.2MB/s 
     |████████████████████████████████| 317kB 51.2MB/s 
     |████████████████████████████████| 32.4MB 110kB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 225kB 55.6MB/s 
     |████████████████████████████████| 460kB 50.5MB/s 
     |████████████████████████████████| 215kB 58.9MB/s 
     |████████████████████████████████| 51

In [10]:
# Download model, config.json, and vocoder
!gdown --id 1GMZlB9kwXbji-TSOsxGlPDDUZ7V-k-JP -O tts_model.pth.tar
!gdown --id 1CpnesSgHBUFxK3tiUl_ExprnL8LAE0lc -O config.json
!gdown --id 1D-ImP_0REVKJ9qlco5JOwxgTCa9L0Uiq -O scale_stats.npy

Downloading...
From: https://drive.google.com/uc?id=1GMZlB9kwXbji-TSOsxGlPDDUZ7V-k-JP
To: /content/TTS/TTS/bin/tts_model.pth.tar
302MB [00:02, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CpnesSgHBUFxK3tiUl_ExprnL8LAE0lc
To: /content/TTS/TTS/bin/config.json
100% 7.66k/7.66k [00:00<00:00, 6.69MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D-ImP_0REVKJ9qlco5JOwxgTCa9L0Uiq
To: /content/TTS/TTS/bin/scale_stats.npy
100% 10.5k/10.5k [00:00<00:00, 26.4MB/s]


In [12]:
# pull the trigger
# !python train_glow_tts.py --config_path /content/config.json
!python /content/TTS/TTS/bin/train_glow_tts.py --config_path /content/TTS/TTS/bin/config.json --restore_path /content/TTS/TTS/bin/tts_model.pth.tar

2021-01-08 07:56:08.690968: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
 > Using CUDA:  True
 > Number of GPUs:  1
Traceback (most recent call last):
  File "/content/TTS/TTS/bin/train_glow_tts.py", line 604, in <module>
    check_config_tts(c)
  File "/usr/local/lib/python3.6/dist-packages/TTS-0.0.6+9cf474a-py3.6-linux-x86_64.egg/TTS/tts/utils/generic_utils.py", line 258, in check_config_tts
    check_argument('external_speaker_embedding_file', c, restricted=c['use_external_speaker_embedding_file'], val_type=str)
KeyError: 'use_external_speaker_embedding_file'
